In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.3, 0.3, 0.3), (0.3, 0.3, 0.3))])

batch_size = 8

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Hamdinn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
hamdi=Hamdinn()

hamdiparam=hamdi.parameters()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(hamdiparam, lr=0.001, momentum=0.9)

In [4]:
for epoch in range(4): 

    running_loss = 0.0
    for i, d in enumerate(trainloader, 0):
        inputs, labels = d

        optimizer.zero_grad()
        outputs = hamdi(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 3000 == 1999:    
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 3000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.396
[1,  5000] loss: 1.637
[2,  2000] loss: 0.937
[2,  5000] loss: 1.320
[3,  2000] loss: 0.807
[3,  5000] loss: 1.173
[4,  2000] loss: 0.717
[4,  5000] loss: 1.082
Finished Training


In [5]:
torch.save(hamdi.state_dict(), './hamdinet.pth')

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = hamdi(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 62 %


In [7]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = hamdi(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 66.9 %
Accuracy for class: car   is 76.2 %
Accuracy for class: bird  is 51.7 %
Accuracy for class: cat   is 47.5 %
Accuracy for class: deer  is 53.4 %
Accuracy for class: dog   is 37.0 %
Accuracy for class: frog  is 68.0 %
Accuracy for class: horse is 73.4 %
Accuracy for class: ship  is 75.4 %
Accuracy for class: truck is 73.3 %


In [ ]:
#BY:Hamdi abderrahmene
#inspired by:https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
